Исходные данные - https://www.kaggle.com/c/sberbank-russian-housing-market/data

In [1]:
import numpy as np
import pandas as pd
import random

import matplotlib as plt
import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score

#normalizing data
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb

#metrics
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from scipy.stats import spearmanr

#model parameters selection
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

#imputer
from sklearn.impute import SimpleImputer

import pickle
#import dill
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

warnings.filterwarnings('ignore')
plt.rcParams.update({'font.size': 14})
pd.set_option('display.max_columns', 500)

In [2]:
TRAIN_DATASET_PATH = './project_data/train.csv'
TEST_DATASET_PATH = './project_data/test.csv'
#MACRO_INFO_PATH = './project_data/macro.csv'
#SAMPLE_SUBMISSION = './project_data/sample_submission.csv'
PREPARED_X_TRAIN_DATASET_PATH = './project_data/X_train.csv'
PREPARED_Y_TRAIN_DATASET_PATH = './project_data/y_train.csv'
PREPARED_X_VALID_DATASET_PATH = './project_data/X_valid.csv'
PREPARED_Y_VALID_DATASET_PATH = './project_data/y_valid.csv'

In [3]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    """
    Функция выводит основные метрики для оценки модели: R2, MAE, MSE.
    
    """
    print("Train:" + "\n" + 
        "R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)) + "\n" +
        "MAE:\t" + str(round(mae(test_true_values, test_pred_values), 3)) + "\n" +
        "MSE:\t" + str(round(mse(test_true_values, test_pred_values), 3)))
    print("Test:" + "\n" + 
        "R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)) + "\n" +
        "MAE:\t" + str(round(mae(test_true_values, test_pred_values), 3)) + "\n" +
        "MSE:\t" + str(round(mse(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(12,8))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

In [4]:
def find_parameters(pipe, params):
    
    rs = GridSearchCV(pipe, param_grid=params, scoring='r2', cv=5)  # Тут же делаем кросс-валидацию
    rs.fit(X_train, y_train)  # y_train.values.ravel()
    
    print(f"Model: {pipe['regressor']},\n params: {rs.best_params_},\n best_score: {rs.best_score_}\n\n")

In [5]:
def find_parameters_rand_grid(pipe, params):
    
    cv = KFold(n_splits=3, random_state=21, shuffle=True)
    # Аналог gridsearchCV, с той разницей, что прорабатывает рандомные параметры в сетке n_iter раз (по умолчанию 10).
    rs = RandomizedSearchCV(pipe, params, scoring='r2', cv=cv, n_iter=15, n_jobs=-1)
    rs.fit(X_train, y_train)  # y_train.values.ravel()
    
    print(f"Model: {pipe['regressor']},\n params: {rs.best_params_},\n best_score: {rs.best_score_}\n\n")

In [6]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    
class FitMedianNones(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X.loc[X[self.key].isnull(), self.key] = X[self.key].median()
        return X[[self.key]]

class FitMedianNonesZeros(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X.loc[(X[self.key] == 0) | (X[self.key].isnull()), self.key] = X[self.key].median()
        return X[[self.key]]

In [7]:
train_df_all = pd.read_csv(TRAIN_DATASET_PATH)
train_df_all.tail(3)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [8]:
test_df_all = pd.read_csv(TEST_DATASET_PATH)
test_df_all.tail(3)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_av

In [9]:
#macro_df_all = pd.read_csv(MACRO_INFO_PATH)
#macro_df_all.tail(3)

In [10]:
#sample_df = pd.read_csv(SAMPLE_SUBMISSION)
#sample_df.tail(3)

Упрощаю датасет, т.к. для курсового не требуется высокая точность

In [11]:
# Отбросим все транзакции по квартирам, кроме последних
train_df_all = train_df_all.drop_duplicates(subset=train_df_all.columns[2:-1], keep='last')
test_df_all = test_df_all.drop_duplicates(subset=test_df_all.columns[2:], keep='last')

In [12]:
# calculate spearman's correlation
coeff_list=[]
for feature in test_df_all.columns[2:]:  
    corr, _ = spearmanr(train_df_all[feature], train_df_all['price_doc'])
    coeff_list.append([feature, corr])
coeff_list = np.array(coeff_list)

In [13]:
coeff_list[coeff_list[:, 1].argsort()]

array([['prom_part_2000', '-0.00319641767714174'],
       ['green_part_500', '-0.003579577986004878'],
       ['cafe_count_500_price_high', '-0.004167871763949121'],
       ['green_part_1000', '-0.004653647398863564'],
       ['cemetery_km', '-0.0052735883659532495'],
       ['ID_metro', '-0.006963718192420888'],
       ['water_km', '-0.014440858833883304'],
       ['oil_chemistry_raion', '-0.02580801314938736'],
       ['green_zone_km', '-0.028553653063391318'],
       ['green_part_1500', '-0.03281578546172717'],
       ['prom_part_1500', '-0.036716921671935014'],
       ['railroad_1line', '-0.03900241509123833'],
       ['prom_part_1000', '-0.039115383962467325'],
       ['big_market_km', '-0.054029729823956645'],
       ['mkad_km', '-0.057766038814699286'],
       ['water_1line', '-0.058819069782471936'],
       ['indust_part', '-0.05935085381633111'],
       ['prom_part_500', '-0.06144695530564172'],
       ['green_part_2000', '-0.07148088464842502'],
       ['incineration_km', '-0

In [14]:
# Оставляем ключевые признаки
basic_features=['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', \
               'sub_area', 'metro_km_walk', 'metro_km_avto', 'build_count_before_1920', 'build_count_1921-1945', \
               'build_count_1946-1970', 'build_count_1971-1995', 'build_count_after_1995', 'raion_build_count_with_builddate_info', \
               'green_zone_km', 'park_km', 'water_treatment_km', 'kremlin_km', 'full_all', 'young_all', 'work_all', 'ekder_all', \
               'green_zone_part', 'indust_part', 'oil_chemistry_raion', 'big_market_raion']
target=['price_doc']

In [15]:
train_df = train_df_all[basic_features] 
train_df[target] = train_df_all[target]

test_df = test_df_all[basic_features] 

train_df.tail(3)

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,sub_area,metro_km_walk,metro_km_avto,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,raion_build_count_with_builddate_info,green_zone_km,park_km,water_treatment_km,kremlin_km,full_all,young_all,work_all,ekder_all,green_zone_part,indust_part,oil_chemistry_raion,big_market_raion,price_doc
30468,45,NaN,10.0,20.0,1.0,NaN,1.0,1.0,1.0,Poselenie Vnukovskoe,1.722233,1.722233,NaN,NaN,NaN,NaN,NaN,NaN,0.427248,4.566595,16.78463,21.905792,17790,574,2566,861,0.496315,0.007122,no,no,6970959
30469,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,2.0,Obruchevskoe,2.047312,2.047312,0.0,0.0,84.0,36.0,66.0,186.0,0.227547,1.772506,11.39752,13.622569,83844,9414,51445,17757,0.167526,0.093443,no,no,13500000
30470,43,28.0,1.0,9.0,1.0,1968.0,2.0,6.0,2.0,Novogireevo,0.454650,0.454650,1.0,2.0,220.0,66.0,14.0,303.0,0.559699,0.848766,10.45101,11.812614,72131,13523,56908,24130,0.063755,0.038693,no,no,5600000


In [16]:
train_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29363 entries, 0 to 30470
Data columns (total 31 columns):
full_sq                                  29363 non-null int64
life_sq                                  23736 non-null float64
floor                                    29219 non-null float64
max_floor                                20492 non-null float64
material                                 20492 non-null float64
build_year                               16730 non-null float64
num_room                                 20492 non-null float64
kitch_sq                                 20492 non-null float64
state                                    16681 non-null float64
sub_area                                 29363 non-null object
metro_km_walk                            29338 non-null float64
metro_km_avto                            29363 non-null float64
build_count_before_1920                  24769 non-null float64
build_count_1921-1945                    24769 non-null float6

In [17]:
train_df.describe()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,metro_km_walk,metro_km_avto,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,raion_build_count_with_builddate_info,green_zone_km,park_km,water_treatment_km,kremlin_km,full_all,young_all,work_all,ekder_all,green_zone_part,indust_part,price_doc
count,29363.000000,23736.000000,29219.000000,20492.000000,20492.000000,1.673000e+04,20492.000000,20492.000000,16681.000000,29338.000000,29363.000000,24769.000000,24769.000000,24769.000000,24769.000000,24769.000000,24769.000000,29363.000000,29363.000000,29363.000000,29363.000000,2.936300e+04,29363.000000,29363.000000,29363.000000,29363.000000,29363.000000,2.936300e+04
mean,54.243333,34.368765,7.618707,12.513078,1.837351,3.078592e+03,1.914113,6.470818,2.115221,3.518184,3.654876,18.962130,26.856635,143.318543,81.177682,61.558117,331.873108,0.298852,3.057457,11.281508,15.883846,1.477752e+05,11384.668120,54725.170419,19619.563600,0.218226,0.118342,7.193445e+06
std,38.581466,52.604955,5.302426,6.746957,1.487555,1.550140e+05,0.853612,28.525241,0.878348,5.742892,5.773544,61.026153,62.725149,125.365055,57.686847,114.919304,277.278079,0.289864,3.955670,6.986316,8.426923,2.868409e+05,8269.522013,37307.842120,13098.506748,0.174893,0.118919,4.829535e+06
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.003737,0.274139,0.072897,2.546000e+03,365.000000,1633.000000,548.000000,0.001879,0.000000,1.000000e+05
25%,38.000000,20.000000,3.000000,9.000000,1.000000,1.967000e+03,1.000000,1.000000,1.000000,0.946477,1.036568,0.000000,0.000000,28.000000,39.000000,14.000000,186.000000,0.103756,0.961911,5.320048,10.342305,3.207100e+04,4203.000000,19960.000000,7589.000000,0.065409,0.017647,4.800000e+06
50%,49.000000,30.000000,6.000000,12.000000,1.000000,1.979000e+03,2.000000,6.000000,2.000000,1.667034,1.762496,0.000000,2.000000,135.000000,71.000000,24.000000,282.000000,0.214625,1.793574,10.378040,14.601792,8.521900e+04,11205.000000,52793.000000,20184.000000,0.167526,0.072158,6.300000e+06
75%,63.000000,43.000000,11.000000,17.000000,2.000000,2.005000e+03,2.000000,9.000000,3.000000,3.480682,3.759210,3.000000,20.000000,216.000000,125.000000,57.000000,400.000000,0.414903,3.234348,16.916350,20.549464,1.251110e+05,15500.000000,78670.000000,29431.000000,0.336177,0.194489,8.400000e+06
max,5326.000000,7478.000000,77.000000,117.000000,6.000000,2.005201e+07,19.000000,2014.000000,33.000000,59.267984,74.905763,371.000000,382.000000,845.000000,246.000000,799.000000,1680.000000,1.982448,47.351538,47.591240,70.738769,1.716730e+06,40692.000000,161290.000000,57086.000000,0.852923,0.521867,1.111111e+08


### Предобработка переменных

In [18]:
# Нормализуем число домов разных годов
build_counts = ['build_count_before_1920', 'build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995', 'build_count_after_1995']

for counts in build_counts:
    train_df[counts] = train_df[counts] / train_df['raion_build_count_with_builddate_info']
    test_df[counts] = test_df[counts] / test_df['raion_build_count_with_builddate_info']

In [19]:
# Нормализуем число домов разных годов
population_types = ['young_all', 'work_all', 'ekder_all']

for type_ in population_types:
    train_df[type_] = train_df[type_] / train_df['full_all']
    test_df[type_] = test_df[type_] / test_df['full_all']

In [20]:
train_df.tail(3)

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,sub_area,metro_km_walk,metro_km_avto,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,raion_build_count_with_builddate_info,green_zone_km,park_km,water_treatment_km,kremlin_km,full_all,young_all,work_all,ekder_all,green_zone_part,indust_part,oil_chemistry_raion,big_market_raion,price_doc
30468,45,NaN,10.0,20.0,1.0,NaN,1.0,1.0,1.0,Poselenie Vnukovskoe,1.722233,1.722233,NaN,NaN,NaN,NaN,NaN,NaN,0.427248,4.566595,16.78463,21.905792,17790,0.032265,0.144238,0.048398,0.496315,0.007122,no,no,6970959
30469,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,2.0,Obruchevskoe,2.047312,2.047312,0.0000,0.000000,0.451613,0.193548,0.354839,186.0,0.227547,1.772506,11.39752,13.622569,83844,0.112280,0.613580,0.211786,0.167526,0.093443,no,no,13500000
30470,43,28.0,1.0,9.0,1.0,1968.0,2.0,6.0,2.0,Novogireevo,0.454650,0.454650,0.0033,0.006601,0.726073,0.217822,0.046205,303.0,0.559699,0.848766,10.45101,11.812614,72131,0.187478,0.788953,0.334530,0.063755,0.038693,no,no,5600000


In [21]:
continuous_columns_nones_zeros=['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', \
                                'kitch_sq', 'metro_km_walk', 'metro_km_avto', 'green_zone_km', 'park_km', \
                                'water_treatment_km', 'kremlin_km', 'full_all', 'young_all', 'work_all', 'ekder_all']
#other_columns = ['sub_area']  # можно использовать для ferature engeneering потом
categorical_columns = ['oil_chemistry_raion', 'big_market_raion']  # Использую также для stratify
continuous_columns_nones = ['build_count_before_1920', 'build_count_1921-1945', 'state', \
                            'build_count_1946-1970', 'build_count_1971-1995', 'build_count_after_1995', \
                            'raion_build_count_with_builddate_info', 'green_zone_part', 'indust_part']

In [22]:
feature_names = continuous_columns_nones_zeros + continuous_columns_nones + categorical_columns

y = train_df[target]
X = train_df[feature_names]

In [23]:
# Использую параметр stratify для бинарных показателей, чтобы обеспечить равномерное распределение 1 и 0
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, stratify=train_df[categorical_columns], random_state=42)

In [24]:
#save train
X_train.to_csv(PREPARED_X_TRAIN_DATASET_PATH, index=False, encoding='utf-8')
y_train.to_csv(PREPARED_Y_TRAIN_DATASET_PATH, index=False, encoding='utf-8')
#save valid
X_valid.to_csv(PREPARED_X_VALID_DATASET_PATH, index=False, encoding='utf-8')
y_valid.to_csv(PREPARED_Y_VALID_DATASET_PATH, index=False, encoding='utf-8')

In [25]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))  # Увеличивает число столбцов
            ])
    final_transformers.append((cat_col, cat_transformer))

# StandardScaler() - https://stackoverflow.com/questions/51459406/how-to-apply-standardscaler-in-pipeline-in-scikit-learn-sklearn
for cont_col in continuous_columns_nones_zeros:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('preparing', FitMedianNonesZeros(key=cont_col))
                #('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))
    
for cont_col in continuous_columns_nones:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('preparing', FitMedianNones(key=cont_col))
                #('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Однако для тестового проекта оставлю только самые базовые фичи, чтобы было легче реализовать фронт.

In [26]:
continuous_columns_nones_zeros=['full_sq', 'build_year', 'num_room']

In [27]:
final_transformers = list()

# StandardScaler() - https://stackoverflow.com/questions/51459406/how-to-apply-standardscaler-in-pipeline-in-scikit-learn-sklearn
for cont_col in continuous_columns_nones_zeros:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('preparing', FitMedianNonesZeros(key=cont_col))
                #('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [28]:
pipeline_xgb = Pipeline([
    ('features', feats),
    ('regressor', xgb.XGBRegressor(random_state = 24)),
])

Т.к. нам выгодно, чтобы деревья были максимально не похожи друг на друга, то нужно, чтобы они обучались не на всей выборке, а на её части и эти части были разными. За это отвечает параметр subsample (по умолчанию 1).

Также поможет параметр colsample_bytree - кол-во фич на дерево (have a range of (0, 1]). Смысл в том, что каждое дерево обучается на на всех фичах, а на сэмпле.

Он также помогает снизить переобучаемость (см. https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

In [29]:
params_xgb = dict(regressor__n_estimators=[300, 500, 800],
                 regressor__max_depth=[2, 3, 4],
                  regressor__learning_rate=[0.01, 0.05, 0.1],
                 regressor__subsample=[0.7, 0.8, 0.6],
                 regressor__colsample_bytree=[0.7, 0.8, 0.6])

In [30]:
#%%time
#find_parameters(pipeline_xgb, params_xgb)

In [31]:
%%time
find_parameters_rand_grid(pipeline_xgb, params_xgb)

Model: XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             objective='reg:squarederror', random_state=24, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=None,
             tree_method=None, validate_parameters=None, verbosity=None),
 params: {'regressor__subsample': 0.7, 'regressor__n_estimators': 800, 'regressor__max_depth': 3, 'regressor__learning_rate': 0.01, 'regressor__colsample_bytree': 0.8},
 best_score: 0.440027712101765


Wall time: 1min


In [32]:
# Подставим параметры, полученные при подборе gridsearch
pipeline_xgb = Pipeline([
    ('features',feats),
    ('regressor', xgb.XGBRegressor(max_depth=5, n_estimators=800, learning_rate=0.1, subsample=0.7, colsample_bytree=0.7, random_state = 24)),
])

pipeline_xgb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('full_sq',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='full_sq')),
                                                                 ('preparing',
                                                                  FitMedianNonesZeros(key='full_sq'))],
                                                          verbose=False)),
                                                ('build_year',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='build_year')),
                            

In [33]:
y_train_preds = pipeline_xgb.predict(X_train)

In [34]:
r2(y_train, y_train_preds)

0.6646771621740493

Сохраним модель (пайплайн)

In [35]:
with open("pipeline_xgb.pkl", "wb") as f:
    pickle.dump(pipeline_xgb, f)